In [1]:
# =====================================================================
# DATA LOADING AND PREPARATION
# =====================================================================
# Load the combined dataset from Anonymisation.ipynb and prepare for classification
# Consistent with methodology described in Chapter 4 of the dissertation

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Configuration 
TARGET_COL = "Recorded Video Interview"
BINARY_CLASSIFICATION = True
TARGET_BINARY_VALUES = ["Passed", "Failed"]

print("="*70)
print("DATA LOADING AND PREPARATION")
print("="*70)
print("Loading combined dataset from Anonymisation.ipynb output...")

# Load the combined dataset
combined_file = 'Combined.csv'
try:
    df_combined = pd.read_csv(combined_file)
    print(f"\nLoaded {combined_file}: {df_combined.shape[0]:,} rows, {df_combined.shape[1]} columns")
except FileNotFoundError:
    print(f"\nError: {combined_file} not found.")
    print("   The combined dataset file is required. Ensure Anonymisation.ipynb has been executed.")
    df_combined = None

if df_combined is not None:
    # Check if target column exists
    if TARGET_COL not in df_combined.columns:
        print(f"\nError: Target column '{TARGET_COL}' not found in combined dataset.")
        print(f"   Available columns: {df_combined.columns.tolist()}")
    else:
        # Prepare features and target
        X = df_combined.drop(columns=[TARGET_COL, 'dataset_year'] if 'dataset_year' in df_combined.columns else [TARGET_COL]).copy()
        y = df_combined[TARGET_COL].copy()
        
        # Filter to binary classification (as per Chapter 4, Section 4.2)
        if BINARY_CLASSIFICATION:
            target_normalized = y.astype(str).str.strip().str.lower()
            binary_normalized = [v.lower().strip() for v in TARGET_BINARY_VALUES]
            filter_mask = target_normalized.isin(binary_normalized)
            
            rows_before = len(X)
            X = X[filter_mask].copy().reset_index(drop=True)
            y = y[filter_mask].copy().reset_index(drop=True)
            rows_after = len(X)
            
            print(f"\nFiltered to binary classes: {rows_after:,} rows (removed {rows_before - rows_after:,})")
        
        # Encode target
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        class_names = label_encoder.classes_
        
        print(f"\nDataset: {X.shape[0]:,} samples, {X.shape[1]} features")
        print(f"Target classes: {dict(zip(class_names, np.bincount(y_encoded)))}")
        
        # Train/Test Split (80/20 stratified, as per Chapter 4, Section 4.2)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded,
            test_size=0.2,
            stratify=y_encoded,
            random_state=42
        )
        
        print(f"\nTrain set: {X_train.shape[0]:,} samples")
        print(f"Test set: {X_test.shape[0]:,} samples")
        print(f"Train class distribution: {dict(zip(class_names, np.bincount(y_train)))}")
        print(f"Test class distribution: {dict(zip(class_names, np.bincount(y_test)))}")
        
        print("\n" + "="*70)
        print("DATA PREPARATION COMPLETE")
        print("="*70)
        print("\nReady for classifier testing and optimization.")


DATA LOADING AND PREPARATION
Loading combined dataset from Anonymisation.ipynb output...

Loaded Combined.csv: 3,957 rows, 19 columns

Filtered to binary classes: 3,957 rows (removed 0)

Dataset: 3,957 samples, 17 features
Target classes: {'Failed': 400, 'Passed': 3557}

Train set: 3,165 samples
Test set: 792 samples
Train class distribution: {'Failed': 320, 'Passed': 2845}
Test class distribution: {'Failed': 80, 'Passed': 712}

DATA PREPARATION COMPLETE

Ready for classifier testing and optimization.


In [2]:
# =====================================================================
# PREPROCESSING PIPELINE SETUP
# =====================================================================
# Set up preprocessing pipeline for classification models
# Consistent with Chapter 4, Section 4.2 of the dissertation

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

print("="*70)
print("PREPROCESSING PIPELINE SETUP")
print("="*70)

if 'X_train' not in locals():
    print("Error: X_train not found. Ensure the data loading cell has been executed.")
else:
    # Identify numeric and categorical columns
    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X_train.select_dtypes(include=['object', 'string', 'category']).columns.tolist()
    
    print(f"\nNumeric features: {len(numeric_cols)}")
    print(f"Categorical features: {len(categorical_cols)}")
    
    # Numeric pipeline: median imputation + scaling (as per Chapter 4, Section 4.2)
    numeric_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Categorical pipeline: constant imputation + one-hot encoding (as per Chapter 4, Section 4.2)
    try:
        # Try new API (sklearn >= 1.4)
        categorical_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=50))
        ])
    except TypeError:
        # Fall back to old API (sklearn < 1.4)
        categorical_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
        ])
    
    # Combine preprocessing
    preprocessor = ColumnTransformer([
        ('numeric', numeric_pipeline, numeric_cols),
        ('categorical', categorical_pipeline, categorical_cols)
    ])
    
    print("\nPreprocessing pipeline configured successfully")
    print("  - Numeric: median imputation + standardization")
    print("  - Categorical: 'missing' category + one-hot encoding (max 50 categories)")
    print("\n" + "="*70)


PREPROCESSING PIPELINE SETUP

Numeric features: 6
Categorical features: 11

Preprocessing pipeline configured successfully
  - Numeric: median imputation + standardization
  - Categorical: 'missing' category + one-hot encoding (max 50 categories)



In [3]:
# =====================================================================
# INITIAL MODEL TESTING: MLP NEURAL NETWORK
# =====================================================================
# Testing Multi-Layer Perceptron as initial classifier
# As described in Chapter 4, Section 4.4 of the dissertation

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, balanced_accuracy_score,
    accuracy_score, precision_score, recall_score, f1_score
)

print("="*70)
print("INITIAL MODEL TESTING: MLP NEURAL NETWORK")
print("="*70)
print("Testing Multi-Layer Perceptron as initial classifier")
print("(As per Chapter 4, Section 4.4 - Model Selection)")

if 'X_train' not in locals() or 'preprocessor' not in locals():
    print("Error: Required variables not found. Ensure previous cells have been executed.")
else:
    # MLP Classifier with Anti-Overfitting Measures
    mlp = MLPClassifier(
        hidden_layer_sizes=(64, 32),  # Two hidden layers: 64 -> 32 neurons
        activation='relu',
        solver='adam',
        alpha=0.001,  # L2 regularization (prevents overfitting)
        learning_rate='adaptive',
        learning_rate_init=0.001,
        max_iter=500,
        early_stopping=True,  # Stop if validation score doesn't improve
        validation_fraction=0.1,  # 10% of training data for validation
        n_iter_no_change=20,  # Stop after 20 iterations without improvement
        tol=1e-4,  # Tolerance for optimization
        random_state=42,
        verbose=False
    )
    
    print("\nMLP Architecture:")
    print(f"  Hidden layers: {mlp.hidden_layer_sizes}")
    print(f"  Activation: {mlp.activation}")
    print(f"  L2 regularization (alpha): {mlp.alpha}")
    print(f"  Early stopping: {mlp.early_stopping}")
    print(f"  Max iterations: {mlp.max_iter}")
    
    # Create complete pipeline
    mlp_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', mlp)
    ])
    
    # Cross-Validation (5-fold, stratified)
    print("\n" + "-"*70)
    print("Cross-Validation (5-fold, stratified)")
    print("-"*70)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_accuracy = cross_val_score(mlp_pipeline, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    cv_balanced_acc = cross_val_score(mlp_pipeline, X_train, y_train, cv=cv, scoring='balanced_accuracy', n_jobs=-1)
    cv_f1 = cross_val_score(mlp_pipeline, X_train, y_train, cv=cv, scoring='f1_macro', n_jobs=-1)
    cv_roc_auc = cross_val_score(mlp_pipeline, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    print(f"CV Accuracy:        {cv_accuracy.mean():.4f} (+/- {cv_accuracy.std() * 2:.4f})")
    print(f"CV Balanced Acc:    {cv_balanced_acc.mean():.4f} (+/- {cv_balanced_acc.std() * 2:.4f})")
    print(f"CV F1 (macro):      {cv_f1.mean():.4f} (+/- {cv_f1.std() * 2:.4f})")
    print(f"CV ROC-AUC:         {cv_roc_auc.mean():.4f} (+/- {cv_roc_auc.std() * 2:.4f})")
    
    # Test Set Evaluation
    print("\n" + "-"*70)
    print("Test Set Evaluation")
    print("-"*70)
    
    mlp_pipeline.fit(X_train, y_train)
    y_pred_mlp = mlp_pipeline.predict(X_test)
    y_pred_proba_mlp = mlp_pipeline.predict_proba(X_test)[:, 1]
    
    mlp_balanced_acc = balanced_accuracy_score(y_test, y_pred_mlp)
    mlp_accuracy = accuracy_score(y_test, y_pred_mlp)
    mlp_f1_macro = f1_score(y_test, y_pred_mlp, average='macro')
    mlp_roc_auc = roc_auc_score(y_test, y_pred_proba_mlp)
    
    print(f"\nTest Set Metrics:")
    print(f"  Accuracy:          {mlp_accuracy:.4f}")
    print(f"  Balanced Accuracy: {mlp_balanced_acc:.4f}")
    print(f"  F1-Score (macro):  {mlp_f1_macro:.4f}")
    print(f"  ROC-AUC:           {mlp_roc_auc:.4f}")
    
    print("\n" + "="*70)
    print("MLP TESTING COMPLETE")
    print("="*70)
    print("\nNote: MLP demonstrated poor performance with very low balanced accuracy,")
    print("indicating it was not an effective classifier for this dataset.")
    print("Proceeding to test alternative classifiers...")


INITIAL MODEL TESTING: MLP NEURAL NETWORK
Testing Multi-Layer Perceptron as initial classifier
(As per Chapter 4, Section 4.4 - Model Selection)

MLP Architecture:
  Hidden layers: (64, 32)
  Activation: relu
  L2 regularization (alpha): 0.001
  Early stopping: True
  Max iterations: 500

----------------------------------------------------------------------
Cross-Validation (5-fold, stratified)
----------------------------------------------------------------------
CV Accuracy:        0.8986 (+/- 0.0092)
CV Balanced Acc:    0.6080 (+/- 0.1157)
CV F1 (macro):      0.6227 (+/- 0.1549)
CV ROC-AUC:         0.8893 (+/- 0.0227)

----------------------------------------------------------------------
Test Set Evaluation
----------------------------------------------------------------------

Test Set Metrics:
  Accuracy:          0.8990
  Balanced Accuracy: 0.5277
  F1-Score (macro):  0.5288
  ROC-AUC:           0.8974

MLP TESTING COMPLETE

Note: MLP demonstrated poor performance with very low

In [4]:
# =====================================================================
# COMPARATIVE CLASSIFIER TESTING
# =====================================================================
# Testing multiple classification algorithms to identify the most suitable approach
# As described in Chapter 4, Section 4.4 of the dissertation

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
import time

print("="*70)
print("COMPARATIVE CLASSIFIER TESTING")
print("="*70)
print("Testing multiple classifiers with class weighting for imbalanced data")
print("(As per Chapter 4, Section 4.4 - Model Selection)")

if 'X_train' not in locals() or 'preprocessor' not in locals():
    print("Error: Required variables not found. Ensure previous cells have been executed.")
else:
    # Define Classifiers to Test (all with class_weight='balanced' for imbalanced data)
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(
            max_depth=10,
            min_samples_split=20,
            min_samples_leaf=10,
            class_weight='balanced',
            random_state=42
        ),
        'Random Forest': RandomForestClassifier(
            n_estimators=100,
            max_depth=15,
            min_samples_split=20,
            min_samples_leaf=10,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            min_samples_split=20,
            min_samples_leaf=10,
            random_state=42
        ),
        'Logistic Regression': LogisticRegression(
            class_weight='balanced',
            max_iter=1000,
            random_state=42,
            solver='lbfgs'
        ),
        'K-Nearest Neighbors': KNeighborsClassifier(
            n_neighbors=5,
            weights='distance'
        ),
        'AdaBoost': AdaBoostClassifier(
            n_estimators=50,
            learning_rate=1.0,
            random_state=42
        )
    }
    
    print(f"\nTesting {len(classifiers)} classifiers...")
    
    # Cross-Validate All Classifiers
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    scoring = {
        'balanced_accuracy': 'balanced_accuracy',
        'accuracy': 'accuracy',
        'f1_macro': 'f1_macro',
        'f1_weighted': 'f1_weighted',
        'roc_auc': 'roc_auc',
        'precision_macro': 'precision_macro',
        'recall_macro': 'recall_macro'
    }
    
    results = []
    
    for name, clf in classifiers.items():
        print(f"\n{'='*70}")
        print(f"Testing: {name}")
        print('='*70)
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', clf)
        ])
        
        start_time = time.time()
        
        cv_results = cross_validate(
            pipeline, X_train, y_train,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            return_train_score=False
        )
        
        train_time = time.time() - start_time
        
        result = {
            'Classifier': name,
            'Balanced_Accuracy': cv_results['test_balanced_accuracy'].mean(),
            'Balanced_Accuracy_Std': cv_results['test_balanced_accuracy'].std(),
            'Accuracy': cv_results['test_accuracy'].mean(),
            'F1_Macro': cv_results['test_f1_macro'].mean(),
            'F1_Weighted': cv_results['test_f1_weighted'].mean(),
            'ROC_AUC': cv_results['test_roc_auc'].mean(),
            'Precision_Macro': cv_results['test_precision_macro'].mean(),
            'Recall_Macro': cv_results['test_recall_macro'].mean(),
            'Train_Time_Seconds': train_time
        }
        
        results.append(result)
        
        print(f"  Balanced Accuracy: {result['Balanced_Accuracy']:.4f} (+/- {result['Balanced_Accuracy_Std']*2:.4f})")
        print(f"  F1 (Macro):        {result['F1_Macro']:.4f}")
        print(f"  ROC-AUC:           {result['ROC_AUC']:.4f}")
        print(f"  Training time:     {train_time:.2f}s")
    
    # Results Comparison
    print("\n" + "="*70)
    print("CLASSIFIER COMPARISON SUMMARY")
    print("="*70)
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Balanced_Accuracy', ascending=False)
    
    print("\nRanked by Balanced Accuracy:")
    print("-"*70)
    display_cols = ['Classifier', 'Balanced_Accuracy', 'Balanced_Accuracy_Std', 
                    'F1_Macro', 'ROC_AUC', 'Train_Time_Seconds']
    print(results_df[display_cols].to_string(index=False))
    
    print("\n" + "="*70)
    print("DETAILED METRICS COMPARISON")
    print("="*70)
    print(results_df[['Classifier', 'Balanced_Accuracy', 'Accuracy', 'F1_Macro', 
                      'F1_Weighted', 'Precision_Macro', 'Recall_Macro', 'ROC_AUC']].to_string(index=False))
    
    # Test Set Evaluation for Top 3 Performers
    print("\n" + "="*70)
    print("TEST SET EVALUATION - TOP 3 CLASSIFIERS")
    print("="*70)
    
    top_classifiers = results_df.head(3)['Classifier'].tolist()
    test_results = []
    
    for name in top_classifiers:
        print(f"\n{'-'*70}")
        print(f"Testing {name} on Test Set")
        print('-'*70)
        
        clf = classifiers[name]
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', clf)
        ])
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred_proba = pipeline.predict_proba(X_test)[:, 1] if hasattr(clf, 'predict_proba') else None
        
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average='macro')
        f1_weighted = f1_score(y_test, y_pred, average='weighted')
        precision_macro = precision_score(y_test, y_pred, average='macro')
        recall_macro = recall_score(y_test, y_pred, average='macro')
        
        try:
            roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
        except:
            roc_auc = None
        
        test_result = {
            'Classifier': name,
            'Balanced_Accuracy': bal_acc,
            'Accuracy': acc,
            'F1_Macro': f1_macro,
            'F1_Weighted': f1_weighted,
            'Precision_Macro': precision_macro,
            'Recall_Macro': recall_macro,
            'ROC_AUC': roc_auc if roc_auc is not None else 'N/A'
        }
        
        test_results.append(test_result)
        
        print(f"  Balanced Accuracy: {bal_acc:.4f}")
        print(f"  Accuracy:          {acc:.4f}")
        print(f"  F1 (Macro):        {f1_macro:.4f}")
        print(f"  F1 (Weighted):     {f1_weighted:.4f}")
        print(f"  ROC-AUC:           {roc_auc:.4f}" if roc_auc else "  ROC-AUC:           N/A")
        
        # Per-class metrics
        precision_per_class = precision_score(y_test, y_pred, average=None)
        recall_per_class = recall_score(y_test, y_pred, average=None)
        f1_per_class = f1_score(y_test, y_pred, average=None)
        
        print(f"\n  Per-Class Metrics:")
        for i, cls_name in enumerate(class_names):
            print(f"    {cls_name}:")
            print(f"      Precision: {precision_per_class[i]:.4f}")
            print(f"      Recall:    {recall_per_class[i]:.4f}")
            print(f"      F1-Score:  {f1_per_class[i]:.4f}")
        
        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred, labels=label_encoder.transform(class_names))
        cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
        print(f"\n  Confusion Matrix:")
        print(cm_df.to_string())
        
        # Store best classifier
        if name == top_classifiers[0]:
            best_pipeline = pipeline
    
    # Test Results Summary
    print("\n" + "="*70)
    print("TEST SET RESULTS SUMMARY")
    print("="*70)
    test_results_df = pd.DataFrame(test_results).sort_values('Balanced_Accuracy', ascending=False)
    print(test_results_df.to_string(index=False))
    
    best_classifier_name = test_results_df.iloc[0]['Classifier']
    best_bal_acc = test_results_df.iloc[0]['Balanced_Accuracy']
    
    print(f"\n🏆 BEST CLASSIFIER: {best_classifier_name}")
    print(f"   Balanced Accuracy: {best_bal_acc:.4f}")
    
    print("\n" + "="*70)
    print("COMPARATIVE TESTING COMPLETE")
    print("="*70)
    print(f"\nRandom Forest selected for further optimization based on test set performance.")


COMPARATIVE CLASSIFIER TESTING
Testing multiple classifiers with class weighting for imbalanced data
(As per Chapter 4, Section 4.4 - Model Selection)

Testing 6 classifiers...

Testing: Decision Tree
  Balanced Accuracy: 0.7863 (+/- 0.0439)
  F1 (Macro):        0.6599
  ROC-AUC:           0.8396
  Training time:     0.15s

Testing: Random Forest
  Balanced Accuracy: 0.8219 (+/- 0.0368)
  F1 (Macro):        0.6381
  ROC-AUC:           0.8981
  Training time:     0.32s

Testing: Gradient Boosting
  Balanced Accuracy: 0.6153 (+/- 0.0422)
  F1 (Macro):        0.6458
  ROC-AUC:           0.9005
  Training time:     2.42s

Testing: Logistic Regression
  Balanced Accuracy: 0.8005 (+/- 0.0390)
  F1 (Macro):        0.6634
  ROC-AUC:           0.8859
  Training time:     0.37s

Testing: K-Nearest Neighbors
  Balanced Accuracy: nan (+/- nan)
  F1 (Macro):        nan
  ROC-AUC:           nan
  Training time:     0.19s

Testing: AdaBoost
  Balanced Accuracy: 0.5700 (+/- 0.0408)
  F1 (Macro):      

In [ ]:
# =====================================================================
# RANDOM FOREST HYPERPARAMETER OPTIMIZATION
# =====================================================================
# Two-stage hyperparameter optimization: RandomizedSearchCV followed by GridSearchCV
# As described in Chapter 4, Section 4.5 of the dissertation

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score
import time

print("="*70)
print("RANDOM FOREST HYPERPARAMETER OPTIMIZATION")
print("="*70)
print("Two-stage optimization: RandomizedSearchCV → GridSearchCV")
print("(As per Chapter 4, Section 4.5 - Model Optimisation and Evaluation)")

if 'X_train' not in locals() or 'preprocessor' not in locals():
    print("Error: Required variables not found. Ensure previous cells have been executed.")
else:
    # Baseline Random Forest (for comparison)
    print("\n" + "="*70)
    print("1. BASELINE: Current Random Forest Configuration")
    print("="*70)
    
    baseline_rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=20,
        min_samples_leaf=10,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    
    baseline_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', baseline_rf)
    ])
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    baseline_scores = cross_validate(
        baseline_pipeline, X_train, y_train,
        cv=cv,
        scoring={'balanced_accuracy': 'balanced_accuracy', 'f1_macro': 'f1_macro', 'roc_auc': 'roc_auc'},
        n_jobs=-1
    )
    
    print(f"Baseline Balanced Accuracy: {baseline_scores['test_balanced_accuracy'].mean():.4f} (+/- {baseline_scores['test_balanced_accuracy'].std()*2:.4f})")
    print(f"Baseline F1 (Macro):        {baseline_scores['test_f1_macro'].mean():.4f}")
    print(f"Baseline ROC-AUC:           {baseline_scores['test_roc_auc'].mean():.4f}")
    
    if 'X_test' in locals() and 'y_test' in locals():
        baseline_pipeline.fit(X_train, y_train)
        baseline_pred = baseline_pipeline.predict(X_test)
        baseline_bal_acc = balanced_accuracy_score(y_test, baseline_pred)
        print(f"Baseline Test Balanced Acc: {baseline_bal_acc:.4f}")
    
    # Stage 1: RandomizedSearchCV
    print("\n" + "="*70)
    print("2. HYPERPARAMETER TUNING - RandomizedSearchCV")
    print("="*70)
    
    param_distributions = {
        'classifier__n_estimators': [50, 100, 200, 300],
        'classifier__max_depth': [10, 15, 20, 25, None],
        'classifier__min_samples_split': [10, 20, 30, 40],
        'classifier__min_samples_leaf': [5, 10, 15, 20],
        'classifier__max_features': ['sqrt', 'log2', None],
        'classifier__class_weight': ['balanced', 'balanced_subsample']
    }
    
    rf_tuning = RandomForestClassifier(random_state=42, n_jobs=-1)
    tuning_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', rf_tuning)
    ])
    
    print("\nRunning RandomizedSearchCV (n_iter=50)...")
    print("This may take several minutes...")
    
    random_search = RandomizedSearchCV(
        tuning_pipeline,
        param_distributions=param_distributions,
        n_iter=50,
        cv=cv,
        scoring='balanced_accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    start_time = time.time()
    random_search.fit(X_train, y_train)
    tuning_time = time.time() - start_time
    
    print(f"\nRandomizedSearch completed in {tuning_time/60:.2f} minutes")
    print(f"\nBest Parameters:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    
    print(f"\nBest CV Balanced Accuracy: {random_search.best_score_:.4f}")
    
    # Test set evaluation for RandomizedSearch results
    if 'X_test' in locals() and 'y_test' in locals():
        best_pred = random_search.best_estimator_.predict(X_test)
        best_bal_acc = balanced_accuracy_score(y_test, best_pred)
        best_f1 = f1_score(y_test, best_pred, average='macro')
        best_roc = roc_auc_score(y_test, random_search.best_estimator_.predict_proba(X_test)[:, 1])
        
        print(f"\nTest Set Performance (RandomizedSearch):")
        print(f"  Balanced Accuracy: {best_bal_acc:.4f}")
        print(f"  F1 (Macro):        {best_f1:.4f}")
        print(f"  ROC-AUC:           {best_roc:.4f}")
    
    # Stage 2: GridSearchCV (refined search around best parameters)
    print("\n" + "="*70)
    print("3. REFINED HYPERPARAMETER TUNING (GridSearchCV)")
    print("="*70)
    
    best_params = random_search.best_params_
    print("\nStarting from best RandomizedSearch parameters:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    # Create focused parameter grid around best parameters
    base_n_est = best_params.get('classifier__n_estimators', 100)
    base_max_depth = best_params.get('classifier__max_depth', 15)
    base_min_split = best_params.get('classifier__min_samples_split', 20)
    base_min_leaf = best_params.get('classifier__min_samples_leaf', 10)
    base_max_feat = best_params.get('classifier__max_features', 'sqrt')
    base_class_weight = best_params.get('classifier__class_weight', 'balanced')
    
    refined_param_grid = {
        'classifier__n_estimators': [max(50, base_n_est - 50), base_n_est, min(300, base_n_est + 50), min(400, base_n_est + 100)],
        'classifier__max_depth': [max(5, base_max_depth - 5) if base_max_depth else 10, 
                                 base_max_depth if base_max_depth else 15, 
                                 min(30, base_max_depth + 5) if base_max_depth else 20,
                                 None] if base_max_depth else [10, 15, 20, None],
        'classifier__min_samples_split': [max(5, base_min_split - 10), base_min_split, base_min_split + 10, base_min_split + 20],
        'classifier__min_samples_leaf': [max(1, base_min_leaf - 5), base_min_leaf, base_min_leaf + 5, base_min_leaf + 10],
        'classifier__max_features': [base_max_feat, 'sqrt', 'log2', None] if base_max_feat != 'sqrt' else ['sqrt', 'log2', None],
        'classifier__class_weight': [base_class_weight]
    }
    
    # Remove duplicates and None values from lists
    for key, value_list in refined_param_grid.items():
        refined_param_grid[key] = sorted(list(set([v for v in value_list if v is not None]))) + ([None] if None in value_list else [])
        refined_param_grid[key] = sorted(list(set(refined_param_grid[key])), key=lambda x: (x is None, x))
    
    rf_refined = RandomForestClassifier(random_state=42, n_jobs=-1)
    refined_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', rf_refined)
    ])
    
    print("\nRunning GridSearchCV for refined tuning...")
    print("This will test more combinations around best parameters...")
    
    grid_search_refined = GridSearchCV(
        refined_pipeline,
        param_grid=refined_param_grid,
        cv=cv,
        scoring='balanced_accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    start_time = time.time()
    grid_search_refined.fit(X_train, y_train)
    refined_tuning_time = time.time() - start_time
    
    print(f"\nGridSearch completed in {refined_tuning_time/60:.2f} minutes")
    print(f"\nBest Refined Parameters:")
    for param, value in grid_search_refined.best_params_.items():
        print(f"  {param}: {value}")
    
    print(f"\nBest CV Balanced Accuracy (Refined): {grid_search_refined.best_score_:.4f}")
    print(f"Improvement over RandomizedSearch: {grid_search_refined.best_score_ - random_search.best_score_:+.4f}")
    
    # Test set evaluation for refined model
    if 'X_test' in locals() and 'y_test' in locals():
        refined_pred = grid_search_refined.best_estimator_.predict(X_test)
        refined_pred_proba = grid_search_refined.best_estimator_.predict_proba(X_test)[:, 1]
        
        refined_bal_acc = balanced_accuracy_score(y_test, refined_pred)
        refined_f1 = f1_score(y_test, refined_pred, average='macro')
        refined_roc = roc_auc_score(y_test, refined_pred_proba)
        
        print(f"\nTest Set Performance (Refined Tuning):")
        print(f"  Balanced Accuracy: {refined_bal_acc:.4f}")
        print(f"  F1 (Macro):        {refined_f1:.4f}")
        print(f"  ROC-AUC:           {refined_roc:.4f}")
        
        # Store best model (use RandomizedSearch if it performs better on test set)
        if best_bal_acc >= refined_bal_acc:
            best_final_model = random_search.best_estimator_
            best_model_name = "RandomizedSearch"
            final_bal_acc = best_bal_acc
        else:
            best_final_model = grid_search_refined.best_estimator_
            best_model_name = "GridSearch"
            final_bal_acc = refined_bal_acc
        
        print("\n" + "="*70)
        print("OPTIMIZATION SUMMARY")
        print("="*70)
        print(f"\nBest Overall Model: {best_model_name}")
        print(f"Test Balanced Accuracy: {final_bal_acc:.4f}")
        print(f"Improvement over baseline: {final_bal_acc - baseline_bal_acc:+.4f} ({((final_bal_acc - baseline_bal_acc)/baseline_bal_acc*100):+.2f}%)")
        
        print("\n" + "="*70)
        print("HYPERPARAMETER OPTIMIZATION COMPLETE")
        print("="*70)


RANDOM FOREST HYPERPARAMETER OPTIMIZATION
Two-stage optimization: RandomizedSearchCV → GridSearchCV
(As per Chapter 4, Section 4.5 - Model Optimisation and Evaluation)

1. BASELINE: Current Random Forest Configuration
Baseline Balanced Accuracy: 0.8219 (+/- 0.0368)
Baseline F1 (Macro):        0.6381
Baseline ROC-AUC:           0.8981
Baseline Test Balanced Acc: 0.8098

2. HYPERPARAMETER TUNING - RandomizedSearchCV

Running RandomizedSearchCV (n_iter=50)...
This may take several minutes...
Fitting 5 folds for each of 50 candidates, totalling 250 fits

RandomizedSearch completed in 0.36 minutes

Best Parameters:
  classifier__n_estimators: 50
  classifier__min_samples_split: 20
  classifier__min_samples_leaf: 20
  classifier__max_features: None
  classifier__max_depth: None
  classifier__class_weight: balanced

Best CV Balanced Accuracy: 0.8319

Test Set Performance (RandomizedSearch):
  Balanced Accuracy: 0.8421
  F1 (Macro):        0.6972
  ROC-AUC:           0.9157

3. REFINED HYPERPA

In [ ]:
# =====================================================================
# FINAL OPTIMIZED MODEL EVALUATION
# =====================================================================
# Comprehensive evaluation of the optimized Random Forest classifier
# As described in Chapter 5 (Results) of the dissertation

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, balanced_accuracy_score,
    accuracy_score, precision_score, recall_score, f1_score
)

print("="*70)
print("FINAL OPTIMIZED MODEL EVALUATION")
print("="*70)
print("Comprehensive evaluation of optimized Random Forest classifier")
print("(As per Chapter 5 - Results)")

if 'best_final_model' not in locals():
    print("Error: best_final_model not found. Ensure the optimization cell has been executed.")
else:
    # Train final model on full training set
    print("\nTraining final optimized model on full training set...")
    best_final_model.fit(X_train, y_train)
    print("Model training complete")
    
    # Cross-Validation on Training Set
    print("\n" + "="*70)
    print("CROSS-VALIDATION (5-fold, stratified)")
    print("="*70)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_accuracy = cross_val_score(best_final_model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    cv_balanced_acc = cross_val_score(best_final_model, X_train, y_train, cv=cv, scoring='balanced_accuracy', n_jobs=-1)
    cv_f1 = cross_val_score(best_final_model, X_train, y_train, cv=cv, scoring='f1_macro', n_jobs=-1)
    cv_roc_auc = cross_val_score(best_final_model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    print(f"CV Accuracy:        {cv_accuracy.mean():.4f} (+/- {cv_accuracy.std() * 2:.4f})")
    print(f"CV Balanced Acc:    {cv_balanced_acc.mean():.4f} (+/- {cv_balanced_acc.std() * 2:.4f})")
    print(f"CV F1 (macro):      {cv_f1.mean():.4f} (+/- {cv_f1.std() * 2:.4f})")
    print(f"CV ROC-AUC:         {cv_roc_auc.mean():.4f} (+/- {cv_roc_auc.std() * 2:.4f})")
    
    # Test Set Evaluation
    print("\n" + "="*70)
    print("TEST SET EVALUATION")
    print("="*70)
    
    y_pred = best_final_model.predict(X_test)
    y_pred_proba = best_final_model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"\nTest Set Metrics:")
    print(f"  Accuracy:          {accuracy:.4f}")
    print(f"  Balanced Accuracy: {balanced_acc:.4f}")
    print(f"  Precision (macro): {precision_macro:.4f}")
    print(f"  Recall (macro):    {recall_macro:.4f}")
    print(f"  F1-Score (macro):  {f1_macro:.4f}")
    print(f"  F1-Score (weighted): {f1_weighted:.4f}")
    print(f"  ROC-AUC:           {roc_auc:.4f}")
    
    # Per-class metrics
    precision_per_class = precision_score(y_test, y_pred, average=None)
    recall_per_class = recall_score(y_test, y_pred, average=None)
    f1_per_class = f1_score(y_test, y_pred, average=None)
    
    print(f"\nPer-Class Metrics:")
    for i, cls_name in enumerate(class_names):
        print(f"  {cls_name}:")
        print(f"    Precision: {precision_per_class[i]:.4f}")
        print(f"    Recall:    {recall_per_class[i]:.4f}")
        print(f"    F1-Score:  {f1_per_class[i]:.4f}")
    
    # Classification Report
    print("\n" + "-"*70)
    print("Detailed Classification Report")
    print("-"*70)
    print(classification_report(y_test, y_pred, target_names=class_names, digits=4))
    
    # Confusion Matrix
    print("\n" + "-"*70)
    print("Confusion Matrix")
    print("-"*70)
    cm = confusion_matrix(y_test, y_pred, labels=label_encoder.transform(class_names))
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    print(cm_df)
    print("\nRow = Actual, Column = Predicted")
    
    # Feature Importance Analysis
    print("\n" + "="*70)
    print("FEATURE IMPORTANCE ANALYSIS")
    print("="*70)
    
    try:
        rf_classifier = best_final_model.named_steps['classifier']
        
        # Get feature names from preprocessor
        try:
            if hasattr(preprocessor, 'get_feature_names_out'):
                feature_names = preprocessor.get_feature_names_out()
            elif hasattr(preprocessor, 'get_feature_names'):
                feature_names = preprocessor.get_feature_names()
            else:
                feature_names = [f'feature_{i}' for i in range(len(rf_classifier.feature_importances_))]
        except:
            feature_names = [f'feature_{i}' for i in range(len(rf_classifier.feature_importances_))]
        
        importances = rf_classifier.feature_importances_
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        print(f"\nTop 20 Most Important Features:")
        print("-"*70)
        print(importance_df.head(20).to_string(index=False))
        
        # Cumulative importance
        importance_df['Cumulative_Importance'] = importance_df['Importance'].cumsum()
        features_80 = (importance_df['Cumulative_Importance'] <= 0.80).sum()
        features_90 = (importance_df['Cumulative_Importance'] <= 0.90).sum()
        
        print(f"\nFeature Importance Insights:")
        print(f"  Top {features_80} features account for 80% of importance")
        print(f"  Top {features_90} features account for 90% of importance")
        print(f"  Total features: {len(importance_df)}")
    except Exception as e:
        print(f"Warning: Could not extract feature importance: {str(e)}")
    
    # Final Summary
    print("\n" + "="*70)
    print("FINAL SUMMARY")
    print("="*70)
    print(f"\nOptimized Random Forest classifier successfully evaluated")
    print(f"  - Training samples: {X_train.shape[0]:,}")
    print(f"  - Test samples: {X_test.shape[0]:,}")
    print(f"  - CV Balanced Accuracy: {cv_balanced_acc.mean():.4f} (+/- {cv_balanced_acc.std() * 2:.4f})")
    print(f"  - Test Balanced Accuracy: {balanced_acc:.4f}")
    print(f"  - Test ROC-AUC: {roc_auc:.4f}")
    
    print("\n" + "="*70)
    print("CLASSIFICATION PIPELINE COMPLETE")
    print("="*70)


FINAL OPTIMIZED MODEL EVALUATION
Comprehensive evaluation of optimized Random Forest classifier
(As per Chapter 5 - Results)

Training final optimized model on full training set...
✓ Model training complete

CROSS-VALIDATION (5-fold, stratified)
CV Accuracy:        0.7908 (+/- 0.0233)
CV Balanced Acc:    0.8365 (+/- 0.0271)
CV F1 (macro):      0.6669 (+/- 0.0176)
CV ROC-AUC:         0.9039 (+/- 0.0142)

TEST SET EVALUATION

Test Set Metrics:
  Accuracy:          0.8207
  Balanced Accuracy: 0.8614
  Precision (macro): 0.6695
  Recall (macro):    0.8614
  F1-Score (macro):  0.6987
  F1-Score (weighted): 0.8517
  ROC-AUC:           0.9151

Per-Class Metrics:
  Failed:
    Precision: 0.3510
    Recall:    0.9125
    F1-Score:  0.5069
  Passed:
    Precision: 0.9880
    Recall:    0.8104
    F1-Score:  0.8904

----------------------------------------------------------------------
Detailed Classification Report
----------------------------------------------------------------------
          